In [52]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')
from os.path import join
# import config
import numpy as np
import joblib
import seaborn as sns
import matplotlib.pyplot as plt
import analyze_helper
import viz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [53]:
results_dir = analyze_helper.best_results_dir
r, cols_varied, mets = analyze_helper.load_clean_results(results_dir)

100%|██████████| 804/804 [01:05<00:00, 12.25it/s]


experiment varied these params: ['subject', 'num_stories', 'feature_space', 'embedding_layer', 'feature_selection_alpha', 'feature_selection_stability_seeds', 'qa_embedding_model', 'qa_questions_version', 'ndelays']


In [54]:
metric_sort = 'corrs_tune_pc_weighted_mean'

In [47]:
r = r[r.qa_questions_version.isin(['', 'v3_boostexamples'])]
# r = r[r.feature_selection_alpha == -1]
r = r[~r.feature_space.isin(
    ['meta-llama/Llama-2-7b-hf', 'meta-llama/Meta-Llama-3-8B'])]
cols_varied = [c for c in cols_varied if not c in [
    'feature_selection_stability_seeds']]

r = r[r.qa_embedding_model.isin(['', 'ensemble1'])]

# only keep feature selection with stability
r = r[(r.feature_selection_alpha < 0) | (
    r.feature_selection_stability_seeds > 0)]

r.shape

(687, 58)

### Check runs (full grid)

In [51]:
d = r
d = d[d.subject.isin(['S01', 'S02', 'S03'])]
d = d[d.feature_selection_alpha < 0]
d = d.groupby(cols_varied)[['corrs_test_mean']].mean()
cols_top = ['feature_space', 'embedding_layer',
            'qa_embedding_model', 'qa_questions_version', 'feature_selection_alpha']
d = (
    d.pivot_table(index=[c for c in cols_varied if not c in cols_top],
                  columns=cols_top, values='corrs_test_mean', aggfunc='mean')
    .sort_index(axis=1)
)
display(
    d.style
    .background_gradient(cmap='magma', axis=1)
    .format(precision=3)

)

## Check simplified table

In [68]:
d = r
d.num_stories = d.num_stories.replace(-1, 100)
d = d[d.subject.isin(['S01', 'S02', 'S03'])]
d = d[(d.feature_selection_alpha < 0) | (
    d.feature_space_simplified == 'qa_embedder')]
group_cols = ['subject', 'num_stories',
              'feature_space_simplified', 'feature_selection_alpha']
d = d.groupby(group_cols)[mets]
d = d.first().reset_index()
# d = d.groupby(cols_varied)[['corrs_test_mean']].mean()
cols_top = ['feature_space_simplified', 'feature_selection_alpha']
d = (
    d.pivot_table(index=[c for c in group_cols if not c in cols_top],
                  columns=cols_top, values='corrs_test_mean', aggfunc='mean')
    .sort_index(axis=1)
)
display(
    d.style
    .background_gradient(cmap='magma', axis=1)
    .format(precision=3)
)